In [ ]:
# pocitani se sumama, indexy korelace

# in matlab:
# x = [1 2 3];
# y = [2 -3 4];
# xcorr(x, y);

# R_xy[0] = 8
# R_xy[1] = -5
# R_xy[-1] = 5

# in python:

import numpy as np
import plotly.graph_objs as go

# Define the sequences
x = np.array([1, 2, 3])
y = np.array([2, -3, 4])

# Calculate cross-correlation
correlation = np.correlate(x, y, mode='full')

# Create lags for plotting
lags = np.arange(-len(x) + 1, len(y))

# Plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=lags, y=correlation, mode='lines+markers', name='Cross-correlation'))
fig.update_layout(title='Cross-correlation of x and y',
                  xaxis_title='Lags',
                  yaxis_title='Correlation',
                  showlegend=True)
fig.show()

print("Cross-correlation:", correlation)

In [ ]:
# zjisteni uhlu: spocitat korelaci a najit jeji maximum
# difference of arrival, dle kucharky v prezentaci

# [x, fs] = audioread('TwoChannelMic.wav')
# r_xy = xcorr(x(:,1), x(:,2)); # korelace jednotlivych channelu

# hledame lag na zaklade uhlu vzdalenosti mikrofonu od mikrofonovyho pole???? TODO: dohledat co tohle presne znamena
# x_L = size(x,1);
# ax_lg = (-x_L+1):(x_L-1)
# [~,mx_ind] = max(r_xy)
# mx_ind = mx_ind - x_L
# TDO = mx_ind * (1/x_fs)
# c = rychlost zvuku
# d = 0.055 % vzdalenost v metrech
# DOA = asin(TDO*c/d) % v radianech
# DOA / pi * 180  % uhel ve stupnich

import numpy as np
import soundfile as sf
from scipy.signal import correlate
from scipy.constants import speed_of_sound
import plotly.graph_objs as go

# Parameters
d = 0.055  # Distance between microphones in meters
c = speed_of_sound  # Speed of sound in air (~343 m/s at 20°C)

# 1. Load the two-channel audio file
audio_file = './TwoChannelMicrophoneArrayRecording.wav'
x, fs = sf.read(audio_file)

# 2. Calculate cross-correlation between the two channels
r_xy = correlate(x[:, 0], x[:, 1], mode='full')
x_L = len(x)
lags = np.arange(-x_L + 1, x_L)

# 3. Find the lag corresponding to the maximum correlation
mx_ind = np.argmax(r_xy)
mx_ind = mx_ind - x_L + 1

# 4. Calculate TDOA (Time Difference of Arrival)
TDOA = mx_ind / fs
print(f"TDOA: {TDOA:.6f} seconds")

# 5. Calculate DOA (Direction of Arrival) in radians
DOA_rad = np.arcsin(TDOA * c / d)
DOA_deg = np.degrees(DOA_rad)
print(f"DOA: {DOA_rad:.4f} radians, {DOA_deg:.2f} degrees")

# 6. Plot cross-correlation using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=lags, y=r_xy, mode='lines', name='Cross-correlation'))
fig.add_trace(go.Scatter(x=[mx_ind], y=[r_xy[mx_ind + x_L - 1]], mode='markers',
                         marker=dict(size=10, color='red'), name='Max Correlation'))
fig.update_layout(title='Cross-correlation between Channels',
                  xaxis_title='Lags',
                  yaxis_title='Correlation',
                  showlegend=True)
fig.show()


# **Direction of Arrival (DOA) Using Time Difference of Arrival (TDOA)**

## **Overview**
The **Direction of Arrival (DOA)** is the angle from which a sound source is coming relative to an array of microphones. By measuring the **Time Difference of Arrival (TDOA)** between two microphones, we can calculate this angle using the known **speed of sound** and the **distance between the microphones**.

This approach is commonly used in:
- **Acoustic localization systems** (e.g., smart speakers, surveillance systems).
- **Beamforming** in microphone arrays to enhance sound from a specific direction.
- **Robotics and drones** for sound source tracking.

---

## **Conceptual Flow**
1. **Cross-Correlation**: Calculate the cross-correlation between the two microphone signals to find the time lag corresponding to the maximum similarity.
2. **Finding the Lag**: Identify the lag (time shift) at which the cross-correlation is maximum. This lag corresponds to the time difference of arrival (TDOA) of the sound wave between the two microphones.
3. **Calculating TDOA**:
```math
TDOA = \frac{\text{Lag}}{\text{Sampling Rate}}
```
4. **Direction of Arrival (DOA)**:
```math
DOA = \arcsin\left( \frac{TDOA \cdot c}{d} \right)
```
Where:
- \( c \) = Speed of sound in air (~343 m/s at 20°C)
- \( d \) = Distance between microphones

5. **Convert to Degrees**:
```math
DOA_{deg} = DOA \times \frac{180}{\pi}
```

---

## **Cross-Correlation**
Cross-correlation is used to measure the similarity between two signals as a function of the time-lag applied to one of them. It is calculated as:

```math
r_{xy}[\tau] = \sum_{n} x[n] \cdot y[n+\tau]
```

- \( x \) and \( y \) are the signals from the two microphones.
- \( \tau \) is the lag applied to one signal relative to the other.

---

## **Finding the Maximum Correlation**
To find the TDOA, we look for the maximum value in the cross-correlation:
```python
mx_ind = np.argmax(r_xy)
```

This `mx_ind` represents the index at which the cross-correlation is the highest, indicating the lag at which the signals are most similar.

The corresponding lag is then adjusted by the length of the original signal:
```python
mx_ind = mx_ind - x_L + 1
```

---

## **Time Difference of Arrival (TDOA)**
The TDOA is calculated by converting the lag into time:
```python
TDOA = mx_ind / fs
```
Where:
- `mx_ind` = Maximum correlation index (lag)
- `fs` = Sampling rate of the audio

---

## **Calculating Direction of Arrival (DOA)**
Using the TDOA, the DOA can be calculated as:
```python
DOA_rad = np.arcsin(TDOA * c / d)
DOA_deg = np.degrees(DOA_rad)
```

- \( \arcsin \) is used because we are calculating the inverse sine of the ratio.
- The ratio represents the sine of the angle of arrival.

---

## **Visualization and Interpretation**
The **cross-correlation plot** shows the similarity between the two signals as a function of lag. The peak indicates the point of maximum similarity, corresponding to the TDOA.

The **DOA angle** is calculated in both **radians** and **degrees**. This angle indicates the direction from which the sound is coming relative to the microphone pair.

---

## **Applications and Use Cases**
- **Audio beamforming**: Focusing on sounds coming from a specific direction.
- **Sound source localization**: Determining the position of a sound source.
- **Speech enhancement**: Isolating speech from a particular direction in noisy environments.
- **Robotics and drones**: Directional hearing for navigation or source tracking.

---

## **Possible Enhancements**
- **Noise filtering**: Pre-process the audio to improve cross-correlation accuracy.
- **Multi-microphone arrays**: For 2D or 3D localization.
- **Beamforming**: Combining multiple DOAs for more accurate sound source localization.
- **Dynamic environments**: Adapting the speed of sound based on environmental conditions.

---

## **Conclusion**
This implementation uses **cross-correlation** and **TDOA** to estimate the **DOA** of a sound source with a two-microphone setup. It leverages the relationship between time delays and the angle of arrival, providing a robust method for **acoustic source localization**.

---



